In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics, tree
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
# Importing clean, ready to go dataset from previous exercise 2
df = pd.read_csv("../exercise2/log_reg_df_clean.csv")

# X = everything else in the DataFrame minus the target variable
# y = only the target variable
X = df.drop('booking_status', axis=1)
y = df['booking_status']

# split the data into train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
# Fit the decision tree classifier with default hyper-parameters
clf = DecisionTreeClassifier()
model = clf.fit(X, y)

In [ ]:
# pip install dtreeviz
import dtreeviz
viz_model = dtreeviz.model(clf,
                           X_train=X, y_train=y,
                           feature_names=X.columns,
                           target_name='Decision',
                           class_names=["No", "Yes"])

# for viewing in the Jupyter notebook
# Note: this is not practical with large 
# decision trees!
# adjust the scale if you need a bigger picture (for example 2.5)
# viz_model.view(scale=1)

# save decision tree into a vector image (SVG)
# open the SVG-file in a web browser for studying
# Mozilla Firefox might be the most light-weight browser for this!
viz_model.view(scale=1).save("decisiontreetest.svg") 

In [ ]:
from sklearn.tree import export_graphviz
import subprocess
from sklearn import tree

# Export the decision tree to DOT format
export_graphviz(clf, 
                   feature_names=X.columns,  
                   class_names=["No", "Yes"],
                   filled=True, rounded=True, node_ids=True, out_file='tree.dot')



# Convert DOT to SVG
subprocess.call(['dot', '-Tsvg', 'tree.dot', '-o', 'tree.svg'])

# if you only want to visualize without saving to file:

# adjust figsize for a different size
# fig = plt.figure(figsize=(30,12))
# _ = tree.plot_tree(clf, 
#                   feature_names=X.columns,  
#                   class_names=["No", "Yes"],
#                   filled=True, rounded=True, fontsize=12, node_ids=True
#                   )

In [ ]:
model = make_pipeline(StandardScaler(), RandomForestClassifier())
model.fit(X_train, y_train)


In [ ]:
# parameters you can try optimizing for RandomForestClassifier
# criterion , options: "gini", "entropy", "log_loss", default="gini"
# n_estimators: default = 100, the number of decision trees created for the data
# class_weight, options: "balanced", "balanced_subsample"
# ccp_alpha: Complexity parameter, non-negative float, default=0.0. Typical values 0.0 - 0.05.
